In [3]:
import filtlib as fl
import rasterio
import numpy as np

DEM_fn = r"Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\LPM\07092023 Full Run\Masked_Tiles.tif"

def mad_fltr(dem, n=3):
    """Median absolute deviation * factor filter

    Robust outlier removal
    """
    mad, med = mad_calc(dem, return_med=True)
    print('Excluding values outside of range: {1:0.3f} +/- {0}*{2:0.3f}'.format(n, med, mad))
    rangelim = (med - n*mad, med + n*mad)
    out = range_fltr(dem, rangelim)
    return out

def range_fltr(dem, rangelim):
    """Range filter (helper function)
    """
    print('Excluding values outside of range: {0:f} to {1:f}'.format(*rangelim))
    out = np.ma.masked_outside(dem, *rangelim)
    out.set_fill_value(dem.fill_value)
    return out

#Check that input is a masked array
def checkma(a, fix=False):
    #isinstance(a, np.ma.MaskedArray)
    if np.ma.is_masked(a):
        out=a
    else:
        out=np.ma.array(a)
    #Fix invalid values
    #Note: this is not necessarily desirable for writing
    if fix:
        #Note: this fails for datetime arrays! Treated as objects.
        #Note: datetime ma returns '?' for fill value
        from datetime import datetime
        if isinstance(a[0], datetime):
            print("Input array appears to be datetime.  Skipping fix")
        else:
            out=np.ma.fix_invalid(out, copy=False)
    return out

def fast_median(a):
    """Fast median operation for masked array using 50th-percentile
    """
    a = checkma(a)
    #return scoreatpercentile(a.compressed(), 50)
    if a.count() > 0:
        out = np.percentile(a.compressed(), 50)
    else:
        out = np.ma.masked
    return out

def mad_calc(a, axis=None, c=1.4826, return_med=False):
    """Compute normalized median absolute difference
   
    Can also return median array, as this can be expensive, and often we want both med and nmad

    Note: 1.4826 = 1/0.6745
    """
    a = checkma(a)
    #return np.ma.median(np.fabs(a - np.ma.median(a))) / c
    if a.count() > 0:
        if axis is None:

            med = fast_median(a)
            out = fast_median(np.fabs(a - med)) * c
        else:
            med = np.ma.median(a, axis=axis)
            #The expand_dims is necessary for broadcasting
            out = np.ma.median(np.ma.fabs(a - np.expand_dims(med, axis=axis)), axis=axis) * c
    else:
        print("No valid entries for input array")
        out = np.ma.masked
        med = np.ma.masked
    if return_med:
        out = (out, med)
    return out

#Open DEM as numpy array
with rasterio.open(DEM_fn) as src:
    dem = src.read(1, masked=True)
    mad_fltr(dem, n=3)

UnboundLocalError: local variable 'mad' referenced before assignment